Crea una classe di backtesting, del tutto analoga a quella per SMA, che funzioni con il Momentum e Contrarian. Questa classe ha un solo parametro numerico (la finestra) e un parametro booleano (momentum o contrarian).

In [1]:
import pandas as pd
import numpy as np
import tpqoa
from itertools import product
api = tpqoa.tpqoa("oanda.cfg")

In [2]:
class MomentumContrarianBacktester():
    def __init__(self, symbol, start, end, window, ismomentum):
        self._symbol = symbol
        self._start=start
        self._end=end
        self._window=window
        self.results=None #none eh variabile vuota
        self._ismomentum=ismomentum
        self.get_data()
        self.prepare_data(self._window,self._ismomentum)
      
    def get_data(self):
        df=api.get_history(instrument=self._symbol, start=self._start, end=self._end, granularity="H6", price="B").c.to_frame()
        df["logret"]=np.log(df.c/df.c.shift(1))
        df["cumlogret"]=df.logret.sum()
        self._data=df
        self._data.dropna(inplace=True)
        
    def prepare_data(self, window, ismomentum):
        if self._ismomentum is True:
            self._data["position"]=np.where(self._data.logret.rolling(window).mean()>0, +1, -1)
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret

        else:
            self._data["position"]=np.where(self._data.logret.rolling(window).mean()>0, -1, +1)
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret    
        
    def set_parameters(self, window, ismomentum):
        if (self._window is not None) & (self._ismomentum is not None) :
            self.prepare_data(window,ismomentum)
            
    def test_strategy(self):
        df2=self._data.copy()
        df2["cumstrategy"]=df2.strategy.sum()
        performance = df2.cumstrategy.iloc[-1]
        overperformance = performance - df2.cumlogret.iloc[-1]
        maxdrawdown = (df2.strategy.cummax()-df2.strategy.cumsum()).max()
        cumlogret = df2.cumlogret.iloc[-1]
        output = {'Metrics': ['Performance', 'CumLogRet(BuyHold)', 'Overperformance', 'Maxdrawdown'], 
                  'Values': [round(performance, 6), round(cumlogret,6), round(overperformance, 6), round(maxdrawdown, 6)]}
        output = pd.DataFrame(output)
        output = output.set_index('Metrics')
        return output

    
    def optimize_strategy(self, winrange):
        combinazioni = list(product(range(winrange[0],winrange[1]+1),[True,False] ))
        risultati= []
        for comb in combinazioni:
            self.set_parameters(comb[0],comb[1])
            risultati.append(self.test_strategy().iloc[0,0]) 
            
        best_performance = np.max(risultati)
        best_combinazione = combinazioni[np.argmax(risultati)]
        
        AllResults = pd.DataFrame(combinazioni, columns=["window","Momentum"])
        AllResults["performance"]= risultati
        self.results = AllResults
        
        return best_combinazione, best_performance

In [3]:
testClass = MomentumContrarianBacktester(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10", window=3, 
                                         ismomentum=True)

In [4]:
testClass._data.head(20)

,c,logret,cumlogret,position,strategy
time,,,,,
2018-10-10 03:00:00,1.14857,-0.001861,0.005486,-1,NaN
2018-10-10 09:00:00,1.15334,0.004144,0.005486,-1,-0.004144
2018-10-10 15:00:00,1.15184,-0.001301,0.005486,1,0.001301
2018-10-10 21:00:00,1.15599,0.003596,0.005486,1,0.003596
2018-10-11 03:00:00,1.15450,-0.001290,0.005486,1,-0.001290
2018-10-11 09:00:00,1.15579,0.001117,0.005486,1,0.001117
2018-10-11 15:00:00,1.15927,0.003006,0.005486,1,0.003006
2018-10-11 21:00:00,1.16045,0.001017,0.005486,1,0.001017
2018-10-12 03:00:00,1.15861,-0.001587,0.005486,1,-0.001587


In [5]:
testClass.test_strategy()

,Values
Metrics,
Performance,-0.172690
CumLogRet(BuyHold),0.005486
Overperformance,-0.178176
Maxdrawdown,0.228304


In [6]:
testClass.optimize_strategy((1,5))

((5, True), 0.175082)

In [7]:
testClass.results

,window,Momentum,performance
0,1,True,-0.173948
1,1,False,-0.173948
2,2,True,-0.121733
3,2,False,-0.121733
4,3,True,-0.172690
5,3,False,-0.172690
6,4,True,0.084378
7,4,False,0.084378
8,5,True,0.175082
9,5,False,0.175082


In [8]:
testClassWindow10 = MomentumContrarianBacktester(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10", window=10, 
                                         ismomentum=True)

In [9]:
testClassWindow10._data.head(20)

,c,logret,cumlogret,position,strategy
time,,,,,
2018-10-10 03:00:00,1.14857,-0.001861,0.005486,-1,NaN
2018-10-10 09:00:00,1.15334,0.004144,0.005486,-1,-0.004144
2018-10-10 15:00:00,1.15184,-0.001301,0.005486,-1,0.001301
2018-10-10 21:00:00,1.15599,0.003596,0.005486,-1,-0.003596
2018-10-11 03:00:00,1.15450,-0.001290,0.005486,-1,0.001290
2018-10-11 09:00:00,1.15579,0.001117,0.005486,-1,-0.001117
2018-10-11 15:00:00,1.15927,0.003006,0.005486,-1,-0.003006
2018-10-11 21:00:00,1.16045,0.001017,0.005486,-1,-0.001017
2018-10-12 03:00:00,1.15861,-0.001587,0.005486,-1,0.001587


In [10]:
testClassWindow10.test_strategy()

,Values
Metrics,
Performance,0.020090
CumLogRet(BuyHold),0.005486
Overperformance,0.014604
Maxdrawdown,0.088835


In [12]:
testClassWindow10.optimize_strategy((1,10))

((5, True), 0.175082)

In [ ]:
testClassWindow10Contrarian = MomentumContrarianBacktester(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10", window=10, 
                                         ismomentum=False)

In [ ]:
testClassWindow10Contrarian.optimize_strategy()

In [ ]:
testClassWindow10Contrarian.test_strategy()

In [ ]:
#In questo contesto, la strategia momentum è più sensata, perchè il prezzo del dollaro è stato in inerzia, 
#cioè ha mantenuto la tendenzia piiuttosto che oscillato e rimbalzato. 

# Modifica la strategia Momentum/Contrarian:
- prendi in considerazione soltanto il rendimento del periodo precedente invece della media mobile
- prendi in considerazione il volume di scambio: soltanto se il volume di scambio dell
a candela precedente è maggiore del massimo degli N volumi precedenti, allora dai un segnale di acquisto (se rendimento positivo e strategia momentum  oppure se rendimento negativo e strategia contrarian) o di vendita. Se invece il volume non è abbastanza grande, il segnale sarà semplicemente 0, cioè usciamo da eventuali posizioni aperte e non facciamo nulla.

Verifica la strategia e trova il valore di N ottimale.

Opzionale: considera anche le spese di transazione.

DONE avere il logRet del giorno e usare questo al posto della media mobile, potete metterlo il una nuova colonna 
	
avere il volume del giorno, anche questo in una nuova colonna 
	
calcolare il volume MASSIMO degli N giorni precedenti (da ieri fino a N giorni prima) e metterlo in una colonna
	
ogni giorno la posizione è 1 (strategia momentum) se logRet>0 e volume>volume massimo, 
-1 se logRet<0 e volume>volume massimo, 0 altrimenti

momentum: np.where(logret>0 & volume>max(vol)), 1, 0
          np.where(logret<0 & volume<max(vol)), -1, df["position"]
	
poi al solito si sposta la posizione indietro di 1 giorno perché i dati di oggi li sappiamo solo a mercato chiuso.

In [ ]:
#remake 3 volumfriendly


class MomentumContrarianVolumfriendly():
    def __init__(self, symbol, start, end, window, ismomentum):
        self._symbol = symbol
        self._start=start
        self._end=end
        self._window=window
        self.results=None #none eh variabile vuota
        self._ismomentum=ismomentum
        self.get_data()
        self.prepare_data(self._window,self._ismomentum)
      
    def get_data(self):
        df=api.get_history(instrument=self._symbol, start=self._start, end=self._end, granularity="D", price="B")
        df = df[["c", "volume"]]
        df["logret"]=np.log(df.c/df.c.shift(1))
        df["cumlogret"]=df.logret.sum()
        self._data=df
        self._data.dropna(inplace=True)     
                
        
    def prepare_data(self, window, ismomentum):
        self._data.dropna(inplace=True)
        self._data["maxsofar"]=self._data.volume.rolling(window).max().shift(1)
        if ismomentum is True:
            self._data["position"]=np.where((self._data["logret"]>0) & (self._data["volume"]>self._data["maxsofar"]),1, 0)           
            self._data["position"]=np.where((self._data["logret"]<0) & (self._data["volume"]>self._data["maxsofar"]),-1,self._data["position"]) 
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret
        else:
            self._data["position"]=np.where((self._data["logret"]>0) & (self._data["volume"]>self._data["maxsofar"]), -1, 0)
            self._data["position"]=np.where((self._data["logret"]<0) & (self._data["volume"]>self._data["maxsofar"]), +1,self._data["position"])
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret 
        
                  
    def set_parameters(self, window, ismomentum):
        if (self._window is not None) & (self._ismomentum is not None) :
            self.prepare_data(window,ismomentum) 
            
            
    def test_strategy(self):
        df2=self._data.copy()
        df2["cumstrategy"]=df2.strategy.sum()
        performance = df2.cumstrategy.iloc[-1]
        overperformance = performance - df2.cumlogret.iloc[-1]
        maxdrawdown = (df2.strategy.cummax()-df2.strategy.cumsum()).max()
        cumlogret = df2.cumlogret.iloc[-1]
        output = {'Metrics': ['Performance', 'CumLogRet(BuyHold)', 'Overperformance', 'Maxdrawdown'], 
                  'Values': [round(performance, 6), round(cumlogret,6), round(overperformance, 6), round(maxdrawdown, 6)]}
        output = pd.DataFrame(output)
        output = output.set_index('Metrics')
        return output    
    
    
    def optimize_strategy(self, winrange):
        combinazioni = list(product(range(winrange[0],winrange[1]+1),[True,False] ))
        risultati= []
        for comb in combinazioni:
            self.set_parameters(comb[0],comb[1])
            risultati.append(self.test_strategy().iloc[0,0]) 
            
        best_performance = np.max(risultati)
        best_combinazione = combinazioni[np.argmax(risultati)]
        
        AllResults = pd.DataFrame(combinazioni, columns=["window","Momentum"])
        AllResults["performance"]= risultati
        self.results = AllResults
        
        return best_combinazione, best_performance

In [ ]:
testClassVF = MomentumContrarianVolumfriendly(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10", window=2,
                                         ismomentum=True)

In [ ]:
testClassVF._data.head(50)

In [ ]:
testClassVF.test_strategy()

In [ ]:
testClassVF.optimize_strategy((1,10))

In [ ]:
testClassVF.results

In [ ]:
testClassVF._data

In [ ]:
testClassVFCont = MomentumContrarianVolumfriendly(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10", window=10,
                                         ismomentum=False)

In [ ]:
testClassVFCont.optimize_strategy((1,10))

# Opzionale: considera anche le spese di transazione.

In [ ]:
class MomentumContrarianVolumfriendlyCosts(MomentumContrarianVolumfriendly):
    
    # in questi giorni volumegiorno>o<volumemassimo sara sempre falso e verra aperta posizione nulla
    def optimize_strategy(self,n):
        columnnames=["logret", "CSlogret"]
        results10= ["CSlogret"]
        df2=self._data.copy()
        df2["CSlogret"]=df2.logret.cumsum()
        for j in range(1,n+1): #n is window
            df2["max"+str(j)]="" 
            df2["max"+str(j)]= df2.volume.rolling(j).max().shift(1)                                                  
        if self._ismomentum is True:
            for i in range(1,n+1): #loop per creare posizioni e ritorni singoli
                columnnames.append("momentum"+str(i)) 
                df2["position" + str(i)] = np.where((df2["logret"]>0) & (df2["volume"]>df2["max"+str(i)]),1, 0) # momentum strategy
                df2["position" + str(i)] = np.where((df2["logret"]<0) & (df2["volume"]<df2["max"+str(i)]),-1,df2["position"+ str(i)])
                df2["momentum" + str(i)] = df2["position"+str(i)].shift(1)*df2.logret                  
        else:
            for i in range(1,n+1): #loop per creare posizioni e ritorni singoli
                columnnames.append("contrarian"+str(i)) 
                df2["position" + str(i)] = np.where((df2["logret"]>0) & (df2["volume"]>df2["max"+str(i)]),-1, 0) # momentum strategy
                df2["position" + str(i)] = np.where((df2["logret"]<0) & (df2["volume"]<df2["max"+str(i)]),+1,df2["position"+ str(i)])
                df2["contrarian" + str(i)]=df2["position"+str(i)].shift(1)*df2.logret
        for i in range(1,n+1): #loop per calcolare transazioni
            df2["transactions"+str(i)]=""
            df2["transactions"+str(i)]=abs(df2["position"+str(i)]-(df2["position"+str(i)].shift(1)))      
        comm=0.0015
        spread=0.00008/df2.c.mean()
        if self._ismomentum is True:
            for i in range(1,n+1):
                df2["momentum-costs"+str(i)]=""
                columnnames.append("momentum-costs"+str(i))
                df2["momentum-costs"+str(i)]=df2["momentum"+str(i)]-((df2["transactions"+str(i)])*(comm+(spread/2)))
        else:
            for i in range(1,n+1):
                df2["contrarian-costs"+str(i)]=""
                columnnames.append("contrarian-costs"+str(i))
                df2["contrarian-costs"+str(i)]=df2["contrarian"+str(i)]-((df2["transactions"+str(i)])*(comm+(spread/2)))
        self._results=df2  
        return df2[columnnames].sum()

In [ ]:
class MomentumContrarianVolumfriendlyCost(MomentumContrarianVolumfriendly):   
    #def __init__(self):
     #   super().__init__()
      #  self.prepare_data(self._window, self._ismomentum,False)
 
        
    def prepare_data(self, window, ismomentum, countpos=False):
        self._data.dropna(inplace=True)
        self._data["maxsofar"]=self._data.volume.rolling(window).max().shift(1)
        if ismomentum is True:
            self._data["position"]=np.where((self._data["logret"]>0) & (self._data["volume"]>self._data["maxsofar"]),1, 0)           
            self._data["position"]=np.where((self._data["logret"]<0) & (self._data["volume"]>self._data["maxsofar"]),-1,self._data["position"]) 
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret
        else:
            self._data["position"]=np.where((self._data["logret"]>0) & (self._data["volume"]>self._data["maxsofar"]), -1, 0)
            self._data["position"]=np.where((self._data["logret"]<0) & (self._data["volume"]>self._data["maxsofar"]), +1,self._data["position"])
            self._data["strategy"]=self._data.position.shift(1)*self._data.logret 
        if countpos is True:
            self._data["countpos"]=abs(self._data["position"]-(self._data["position"].shift(1)))
        
                  
    def set_parameters(self, window, ismomentum, cost):
        if (self._window is not None) & (self._ismomentum is not None) :
            self.prepare_data(window,ismomentum,cost) 
            
            
    def test_strategy(self, countpos=False):
        df2=self._data.copy()
        comm=0.0015
        spread=0.00008/df2.c.mean()
        df2["cumstrategy"]=df2.strategy.sum()
        performance = df2.cumstrategy.iloc[-1]
        if countpos is True:
            performance = performance - ((df2.countpos.sum())*(comm+(spread/2)))
        overperformance = performance - df2.cumlogret.iloc[-1]
        maxdrawdown = (df2.strategy.cummax()-df2.strategy.cumsum()).max()
        cumlogret = df2.cumlogret.iloc[-1]
        output = {'Metrics': ['Performance', 'CumLogRet(BuyHold)', 'Overperformance', 'Maxdrawdown'], 
                  'Values': [round(performance, 6), round(cumlogret,6), round(overperformance, 6), round(maxdrawdown, 6)]}
        output = pd.DataFrame(output)
        output = output.set_index('Metrics')
        return output    
    
    def optimize_strategy(self, winrange, cost):
        combinazioni = list(product(range(winrange[0],winrange[1]+1),[True,False] ))
        risultati= []
        for comb in combinazioni:
            self.set_parameters(comb[0],comb[1], cost)
            risultati.append(self.test_strategy().iloc[0,0]) 
        best_performance = np.max(risultati)
        best_combinazione = combinazioni[np.argmax(risultati)]
        
        AllResults = pd.DataFrame(combinazioni, columns=["window","Momentum"])
        AllResults["performance"]= risultati
        self.results = AllResults
        
        return best_combinazione, best_performance


In [ ]:
testClassVFCosts10 = MomentumContrarianVolumfriendlyCost(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10",
                                                         window=2, ismomentum=True)

In [ ]:
testClassVFCosts10._data.head(50)

In [ ]:
testClassVFCosts10.test_strategy()

In [ ]:
testClassVFCosts10.optimize_strategy((4,16),True)

In [ ]:
testClassVFCosts10.results

In [ ]:
testClassVFCosts10.set_parameters(5,False,True)

In [ ]:
testClassVFCosts10Contrarian = MomentumContrarianVolumfriendlyCost(symbol = "EUR_USD", start = "2018-10-10", end = "2021-10-10",
                                                            window=6,ismomentum=False)

In [ ]:
testClassVFCosts10Contrarian._data

In [ ]:
testClassVFCosts10Contrarian.test_strategy()

In [ ]:
testClassVFCosts10Contrarian.optimize_strategy((1,10), True)

In [ ]:
testClassVFCosts10Contrarian.optimize_strategy((1,10), False)